In [2]:
%%capture
!pip install -r requirements.txt
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4

In [3]:
from nltk.corpus import stopwords
import snscrape.modules.telegram as telegram
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

In [13]:
channels = ['thebull_crypto']
scraper = telegram.TelegramChannelScraper('coincodecap')
telegram_df = pd.DataFrame()
for i,tel in enumerate(scraper.get_items()):
  if ( i < 100):
    df = pd.DataFrame([{'Date':tel.date,'Content':tel.content}])
    telegram_df = pd.concat([telegram_df,df],axis=0,ignore_index=True)
  else:
    break
telegram_df

,Date,Content
0,2022-09-10 08:29:26+00:00,"$BTC UPDATE Wow, a super pump of 17% with a n..."
1,2022-09-09 22:03:51+00:00,CoinCodeCap Classic pinned a photo
2,2022-09-09 15:36:51+00:00,👆Nothing to say more because these nukes are e...
3,2022-09-09 15:33:46+00:00,"KuCoin, Huobi.pro, Binance#GRT/USDT Take-Profi..."
4,2022-09-09 15:33:46+00:00,"KuCoin, Huobi.pro, Binance#SHIB/USDT Take-Prof..."
...,...,...
95,2022-08-29 16:46:07+00:00,FREE NFT SIGNAL 🔥Follow us on twitter so you d...
96,2022-08-29 15:42:18+00:00,Make sure to comment on our twitter for the an...
97,2022-08-29 14:30:49+00:00,📍#Learning #CCCTradingLesson #49(Falling Wedge...
98,2022-08-29 08:09:19+00:00,$BTC UPDATEBTC is holding at the potential rev...


In [15]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
# current_date =date.today()
def text_cleaning(text):
    if(text!=None):
        stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        return normalized.split()

telegram_df["Cleaned Text"]= telegram_df["Content"].apply(text_cleaning)


In [22]:
def my_predictions(my_sentence, model):
    prediction = model.predict([my_sentence])
    return prediction

result=list()
model = pickle.load(open('./Telegram.pkl', 'rb'))
# telegram_df["Cleaned Text"].apply(lambda x: x.lower())
for i in telegram_df["Cleaned Text"]:
    result.append(my_predictions(str(i),model))
telegram_df["Result"]=result
telegram_df['Result'] = telegram_df['Result'].apply(lambda x: int(x))
telegram_df

,Date,Content,Cleaned Text,Result
0,2022-09-10 08:29:26+00:00,"$BTC UPDATE Wow, a super pump of 17% with a n...","[btc, update, wow, super, pump, 17, nice, gree...",1
1,2022-09-09 22:03:51+00:00,CoinCodeCap Classic pinned a photo,"[coincodecap, classic, pinned, photo]",1
2,2022-09-09 15:36:51+00:00,👆Nothing to say more because these nukes are e...,"[👆nothing, say, nuke, enough, ❤️‍🩹quality, ser...",2
3,2022-09-09 15:33:46+00:00,"KuCoin, Huobi.pro, Binance#GRT/USDT Take-Profi...","[kucoin, huobipro, binancegrtusdt, takeprofit,...",1
4,2022-09-09 15:33:46+00:00,"KuCoin, Huobi.pro, Binance#SHIB/USDT Take-Prof...","[kucoin, huobipro, binanceshibusdt, takeprofit...",1
...,...,...,...,...
95,2022-08-29 16:46:07+00:00,FREE NFT SIGNAL 🔥Follow us on twitter so you d...,"[free, nft, signal, 🔥follow, u, twitter, dont,...",1
96,2022-08-29 15:42:18+00:00,Make sure to comment on our twitter for the an...,"[make, sure, comment, twitter, analysis, https...",1
97,2022-08-29 14:30:49+00:00,📍#Learning #CCCTradingLesson #49(Falling Wedge...,"[📍learning, ccctradinglesson, 49falling, wedge...",1
98,2022-08-29 08:09:19+00:00,$BTC UPDATEBTC is holding at the potential rev...,"[btc, updatebtc, holding, potential, reversal,...",1


In [27]:
telegram_df = telegram_df[telegram_df["Result"]==2]
telegram_df = telegram_df.drop(columns=['Cleaned Text','Result'])

In [22]:
telegram_df.to_csv("Telegram_Predicted.csv")

In [21]:
# !pip freeze > requirements.txt